# Business Problem

Who are the stakeholders in this project? Who will be directly affected by the creation of this project?
A NHL team
What business problem(s) will this Data Science project solve for the organization?
An NHL team is looking to determine who is likely, or more likely to suffer a concussion based on past performance and information. 
What problems are inside the scope of this project?
What problems are outside the scope of this project?
What data sources are available to us?
Eliteprospect_scraper package and a csv of nhl injuries
What is the expected timeline for this project? Are there hard deadlines (e.g. "must be live before holiday season shopping") or is this an ongoing project?
Finish bones of project by 1/17/2023
Do stakeholders from different parts of the company or organization all have the exact same understanding about what this project is and isn't?

In [29]:
!pip install eliteprospect_scraper

In [30]:
import eliteprospect.eliteprospect_scraper as ep

In [76]:
nhl_12_13 = ep.getPlayers('nhl', '2012-13')

C:\Users\Jeff\anaconda3\lib\site-packages\eliteprospect\eliteprospect_scraper.py:86: FutureWarning: The default value of regex will change from True to False in a future version.
  df_players['playername'] = df_players['player'].str.replace(r"\(.*\)","")


In [77]:
nhl_13_14 = ep.getPlayers('nhl', '2013-14')

C:\Users\Jeff\anaconda3\lib\site-packages\eliteprospect\eliteprospect_scraper.py:86: FutureWarning: The default value of regex will change from True to False in a future version.
  df_players['playername'] = df_players['player'].str.replace(r"\(.*\)","")


In [75]:
nhl_12_13.head()

,player,team,gp,g,a,tp,ppg,pim,+/-,link,season,league,playername,position,fw_def
0,Martin St-Louis (RW),Tampa Bay Lightning,48,17,43,60,1.25,14,0,https://www.eliteprospects.com/player/8772/mar...,2012-13,nhl,Martin St-Louis,RW,FW
1,Steven Stamkos (C/W),Tampa Bay Lightning,48,29,28,57,1.19,32,-4,https://www.eliteprospects.com/player/11113/st...,2012-13,nhl,Steven Stamkos,C/W,FW
2,Alexander Ovechkin (LW),Washington Capitals,48,32,24,56,1.17,36,2,https://www.eliteprospects.com/player/4230/ale...,2012-13,nhl,Alexander Ovechkin,LW,FW
3,Sidney Crosby (C),Pittsburgh Penguins,36,15,41,56,1.56,16,26,https://www.eliteprospects.com/player/6146/sid...,2012-13,nhl,Sidney Crosby,C,FW
4,Patrick Kane (RW),Chicago Blackhawks,47,23,32,55,1.17,8,11,https://www.eliteprospects.com/player/9326/pat...,2012-13,nhl,Patrick Kane,RW,FW


In [78]:
nhl_13_14.head()

,player,team,gp,g,a,tp,ppg,pim,+/-,link,season,league,playername,position,fw_def
0,Sidney Crosby (C),Pittsburgh Penguins,80,36,68,104,1.30,46,18,https://www.eliteprospects.com/player/6146/sid...,2013-14,nhl,Sidney Crosby,C,FW
1,Ryan Getzlaf (C),Anaheim Ducks,77,31,56,87,1.13,31,28,https://www.eliteprospects.com/player/8526/rya...,2013-14,nhl,Ryan Getzlaf,C,FW
2,Claude Giroux (W/C),Philadelphia Flyers,82,28,58,86,1.05,46,7,https://www.eliteprospects.com/player/10437/cl...,2013-14,nhl,Claude Giroux,W/C,FW
3,Tyler Seguin (C/RW),Dallas Stars,80,37,47,84,1.05,18,16,https://www.eliteprospects.com/player/23807/ty...,2013-14,nhl,Tyler Seguin,C/RW,FW
4,Corey Perry (RW),Anaheim Ducks,81,43,39,82,1.01,65,32,https://www.eliteprospects.com/player/8541/cor...,2013-14,nhl,Corey Perry,RW,FW


In [70]:
nhl_19_20.shape

(850, 15)

In [71]:
import pandas as pd 

In [72]:
# load in nhl injury data 
injuries = pd.read_csv('nhl_injury.csv', encoding='latin')

In [63]:
injuries.head()

,ï»¿Season,Team,Position,Player,Injury Type,Cap Hit,Chip,Games Missed,Games Missed.1
0,2000/01,Anaheim,F,"Kariya, Paul",Foot,NaN,NaN,16,16
1,2000/01,Anaheim,F,"Leclerc, Mike",Abdominal,NaN,NaN,13,13
2,2000/01,Anaheim,F,"Leclerc, Mike",Knee,NaN,NaN,15,15
3,2000/01,Anaheim,F,"McDonald, Andy",Concussion,NaN,NaN,7,7
4,2000/01,Anaheim,F,"McInnis, Marty",Groin,NaN,NaN,3,3


In [35]:
injuries.columns

Index(['ï»¿Season', 'Team', 'Position', 'Player', 'Injury Type', 'Cap Hit',
       'Chip', 'Games Missed', 'Games Missed.1'],
      dtype='object')

In [36]:
injuries['Injury Type'].unique()

array(['Foot', 'Abdominal', 'Knee', 'Concussion', 'Groin', 'Neck',
       'Dizziness', 'Hand', 'Jaw', 'Headache', 'Ribs', 'Shoulder', 'Back',
       'Ankle', 'Hip', 'Eye', 'Collarbone', 'Thumb', 'Flu', 'Cheek',
       'Hamstring', 'Arm', 'Leg', 'Chest', 'Charley horse', 'Pelvis',
       'Wrist', 'Heel', 'Head', 'Toe', 'Stomach', 'Sternum', 'Finger',
       'Larynx', 'Elbow', 'Sinus', 'Illness', 'Gluteus', 'Thigh', 'Nose',
       'Kidney', 'Hernia', 'Quadriceps', 'Calf', 'Tailbone', 'Mouth',
       'Ear', 'Oblique', 'Shin', 'Bicep', 'Skin infection', 'Spleen',
       'Facial', 'Pneumonia', 'Achilles', 'Gall bladder', 'Dental',
       'Triceps', 'Respiratory', 'Cancer', 'Fatigue', 'Dehydration',
       'Undisclosed', 'Upper body', 'Blood clots', 'Throat',
       'Orbital bone', 'Heart', 'Infection', 'Appendicitis', 'Chickenpox',
       'Ulcerative colitis', 'Lower body', 'Torso', 'Migraine', 'Vertigo',
       'Hepatitis', 'Adductor', 'Bronchitis', 'Sacrum', 'Pubic bone',
       'Rash', '

In [37]:
# filter to only 2020-2021
injuries_19_20 = injuries[injuries['ï»¿Season'] == '2019/20']

In [38]:
# filter to only be injury type of concussion 
concussion_injuries = injuries_19_20[injuries_19_20['Injury Type'] == 'Concussion']

In [39]:
concussion_injuries.head(2)

,ï»¿Season,Team,Position,Player,Injury Type,Cap Hit,Chip,Games Missed,Games Missed.1
17426,2019/20,Boston,G,"Rask, Tuukka",Concussion,7.0,256.097561,3,3
17436,2019/20,Buffalo,F,"Okposo, Kyle",Concussion,6.0,731.707317,10,10


In [40]:
# make new name variable to join between the 2 datasets 
new_names = concussion_injuries['Player'].map(lambda x: x.split(' ')[1]) + ' ' + concussion_injuries['Player'].map(lambda x: x.split(' ')[0])

In [41]:
new_names = [x.replace(',', '') for x in new_names]

In [42]:
concussion_injuries['Name'] = new_names

C:\Users\Jeff\AppData\Local\Temp\ipykernel_18400\2633589963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concussion_injuries['Name'] = new_names


In [43]:
concussion_injuries.head(2)

,ï»¿Season,Team,Position,Player,Injury Type,Cap Hit,Chip,Games Missed,Games Missed.1,Name
17426,2019/20,Boston,G,"Rask, Tuukka",Concussion,7.0,256.097561,3,3,Tuukka Rask
17436,2019/20,Buffalo,F,"Okposo, Kyle",Concussion,6.0,731.707317,10,10,Kyle Okposo


In [44]:
# remove white space from player name
nhl_19_20['playername'] = nhl_19_20['playername'].map(lambda x: x.strip())

In [45]:
# join 2 datasets 
joined_data = nhl_19_20.merge(concussion_injuries['Name'],
                        how='left', left_on='playername',
                        right_on='Name')

In [46]:
# fill in missing values with 0 
joined_data.fillna(0, inplace=True)

In [47]:
# create concussion column 
import numpy as np 

joined_data['Concussion'] = np.where(joined_data['Name'] == 0, 0, 1)

In [48]:
# column I chose - not necessarily the columns you should use 
final_df = joined_data[['player', 'team', 'gp', 'g', 'a', 'tp', 'ppg', 'pim', '+/-', 
 'position', 'fw_def','Concussion']]

In [49]:
final_df.head()

,player,team,gp,g,a,tp,ppg,pim,+/-,position,fw_def,Concussion
0,Leon Draisaitl (C/W),Edmonton Oilers,71,43,67,110,1.55,18,-7,C/W,FW,0
1,Connor McDavid (C),Edmonton Oilers,64,34,63,97,1.52,28,-6,C,FW,0
2,David Pastrnak (RW),Boston Bruins,70,48,47,95,1.36,40,21,RW,FW,0
3,Artemi Panarin (LW),New York Rangers,69,32,63,95,1.38,20,36,LW,FW,0
4,Nathan MacKinnon (C/RW),Colorado Avalanche,69,35,58,93,1.35,12,13,C/RW,FW,0


In [50]:
injuries['ï»¿Season'].unique()

array(['2000/01', '2000/01 (playoffs)', '2001/02', '2001/02 (playoffs)',
       '2002/03', '2002/03 (playoffs)', '2003/04', '2003/04 (playoffs)',
       '2005/06', '2005/06 (playoffs)', '2006/07', '2006/07 (playoffs)',
       '2007/08', '2007/08 (playoffs)', '2008/09', '2008/09 (playoffs)',
       '2009/10', '2009/10 (playoffs)', '2010/11', '2010/11 (playoffs)',
       '2011/12', '2011/12 (playoffs)', '2012/13', '2012/13 (playoffs)',
       '2013/14', '2013/14 (playoffs)', '2014/15', '2014/15 (playoffs)',
       '2015/16', '2015/16 (playoffs)', '2016/17', '2016/17 (playoffs)',
       '2017/18', '2017/18 (playoffs)', '2018/19', '2018/19 (playoffs)',
       '2019/20', '2019/20 (playoffs+)', '2020/21', '2020/21 (playoffs)',
       '2021/22', '2021/22 (playoffs)', '2022/23'], dtype=object)

In [51]:
final_df.head()

,player,team,gp,g,a,tp,ppg,pim,+/-,position,fw_def,Concussion
0,Leon Draisaitl (C/W),Edmonton Oilers,71,43,67,110,1.55,18,-7,C/W,FW,0
1,Connor McDavid (C),Edmonton Oilers,64,34,63,97,1.52,28,-6,C,FW,0
2,David Pastrnak (RW),Boston Bruins,70,48,47,95,1.36,40,21,RW,FW,0
3,Artemi Panarin (LW),New York Rangers,69,32,63,95,1.38,20,36,LW,FW,0
4,Nathan MacKinnon (C/RW),Colorado Avalanche,69,35,58,93,1.35,12,13,C/RW,FW,0


In [52]:
final_df['Concussion'].value_counts()

0    872
1     28
Name: Concussion, dtype: int64

In [53]:
final_df['pim'] = final_df['pim'].astype(int)

C:\Users\Jeff\AppData\Local\Temp\ipykernel_18400\1977981380.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['pim'] = final_df['pim'].astype(int)


In [54]:
final_df.groupby('Concussion')['pim'].mean()

Concussion
0    19.402523
1    16.928571
Name: pim, dtype: float64

In [55]:
final_df['gp'] = final_df['gp'].astype(int)

C:\Users\Jeff\AppData\Local\Temp\ipykernel_18400\3355596331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['gp'] = final_df['gp'].astype(int)


In [57]:
final_df.head()

,player,team,gp,g,a,tp,ppg,pim,+/-,position,fw_def,Concussion
0,Leon Draisaitl (C/W),Edmonton Oilers,71,43,67,110,1.55,18,-7,C/W,FW,0
1,Connor McDavid (C),Edmonton Oilers,64,34,63,97,1.52,28,-6,C,FW,0
2,David Pastrnak (RW),Boston Bruins,70,48,47,95,1.36,40,21,RW,FW,0
3,Artemi Panarin (LW),New York Rangers,69,32,63,95,1.38,20,36,LW,FW,0
4,Nathan MacKinnon (C/RW),Colorado Avalanche,69,35,58,93,1.35,12,13,C/RW,FW,0


In [58]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 0 to 899
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   player      900 non-null    object
 1   team        900 non-null    object
 2   gp          900 non-null    int32 
 3   g           900 non-null    object
 4   a           900 non-null    object
 5   tp          900 non-null    object
 6   ppg         900 non-null    object
 7   pim         900 non-null    int32 
 8   +/-         900 non-null    object
 9   position    900 non-null    object
 10  fw_def      900 non-null    object
 11  Concussion  900 non-null    int32 
dtypes: int32(3), object(9)
memory usage: 80.9+ KB


In [60]:
# final_df.groupby('Concussion')['position'].mean()

In [62]:
final_df.shape

(900, 12)

In [ ]:
final_df.info()

In [ ]:
final_df.head()

In [ ]:
final_df[final_df['Concussion'] == 1]['fw_def'].value_counts(normalize=True)

# Data Understanding

What data is available to us? Where does it live? Do we have the data, or can we scrape/buy/source the data from somewhere else?
Who controls the data sources, and what steps are needed to get access to the data?
What is our target?
What predictors are available to us?
What data types are the predictors we'll be working with?
What is the distribution of our data?
How many observations does our dataset contain? Do we have a lot of data? Only a little?
Do we have enough data to build a model? Will we need to use resampling methods?
How do we know the data is correct? How is the data collected? Is there a chance the data could be wrong?

# Data Preparation

Detecting and dealing with missing values
Data type conversions (e.g. numeric data mistakenly encoded as strings)
Checking for and removing multicollinearity (correlated predictors)
Normalizing our numeric data
Converting categorical data to numeric format through one-hot encodin

# Modeling 

Detecting and dealing with missing values
Data type conversions (e.g. numeric data mistakenly encoded as strings)
Checking for and removing multicollinearity (correlated predictors)
Normalizing our numeric data
Converting categorical data to numeric format through one-hot encodin

# Evaluation

During this step, we'll evaluate the results of our modeling efforts. Does our model solve the problems that we outlined all the way back during step 1? Why or why not?

# 1. Obtain

# 2. Scrub

# 3. Explore

# 4. Model

# 5. Interpret